In [ ]:
# !pip install pyproj

In [42]:
import json
import os

import matplotlib.pyplot as plt
import numpy as np
import pyproj

import mplleaflet

In [43]:
# Load up the geojson data
filename = os.path.join('../mplleaflet/examples', 'data', 'track.geojson')
with open(filename) as f:
    gj = json.load(f)

In [44]:
features = [feat for feat in gj['features'][::10]]
features

[{'type': 'Feature',
  'id': '0',
  'geometry': {'type': 'Point',
   'coordinates': [-71.60285133455481, 42.35857187655945]},
  'properties': {'horizontal_accuracy': 5,
   'vertical_accuracy': 3,
   'course': 55.56803512573242,
   'time': '2014-01-05 20:08:41.342',
   'altitude': 89,
   'speed': 26.90129661560059}},
 {'type': 'Feature',
  'id': '10',
  'geometry': {'type': 'Point',
   'coordinates': [-71.60016015037911, 42.35978095574557]},
  'properties': {'horizontal_accuracy': 5,
   'vertical_accuracy': 3,
   'course': 60.03398132324219,
   'time': '2014-01-05 20:08:51.025',
   'altitude': 92,
   'speed': 26.14530944824219}},
 {'type': 'Feature',
  'id': '20',
  'geometry': {'type': 'Point',
   'coordinates': [-71.59756879343313, 42.36073859087215]},
  'properties': {'horizontal_accuracy': 5,
   'vertical_accuracy': 4,
   'course': 66.33683776855469,
   'time': '2014-01-05 20:09:01.198',
   'altitude': 96,
   'speed': 23.69427490234375}},
 {'type': 'Feature',
  'id': '30',
  'geomet

In [ ]:
xy = np.array([feat['geometry']['coordinates'] for feat in features])
xy

In [ ]:
# Transform the data to EPSG:26986 (Mass. state plane)
proj_in = pyproj.Proj(preserve_units=True, init='epsg:4326', no_defs=True)
crs_out = {'init': 'epsg:26986', 'no_defs': True}
proj_out = pyproj.Proj(preserve_units=True, **crs_out)
xy2 = np.array([pyproj.transform(proj_in, proj_out, c[0], c[1]) for c in xy])
xy2

In [ ]:
# Grab the speed (m/s)
speed = np.array([feat['properties']['speed'] for feat in features])
speed

In [ ]:
# Grab the course. Course is 0 degrees due North, increasing clockwise
course = np.array([feat['properties']['course'] for feat in features])
course

In [ ]:
#### 
angle = np.deg2rad(-course + 90)  # Convert to angle in xy plane
angle

In [ ]:
np.column_stack([np.cos(angle), np.sin(angle)])

In [ ]:
# Normalize the speed to use as the length of the arrows
r = 3*speed / max(speed)
uv = r[:, np.newaxis] * np.column_stack([np.cos(angle), np.sin(angle)])
uv

In [ ]:
xy.shape, xy2.shape, speed.shape, course.shape, angle.shape, uv.shape, r.shape

In [ ]:
import pandas as pd
df = pd.DataFrame(data={'x_in':xy[:,0],
                        'y_in':xy[:,1],
                        'x_out':xy2[:,0],
                        'y_out':xy2[:,1],
                        'speed':speed,
                        'dir':course,
                        'angle':angle,
                        'u':uv[:,0],
                        'v':uv[:,1],
                        'r':r})
df

In [ ]:
# For each point, plot an arrow pointing in the direction of the iPhone's
# course estimate. The arrow length is proportional to the phone's speed
# estimate. For a bigger effect, color each other based on its speed
plt.quiver(xy2[:5,0], xy2[:5,1], uv[:5,0], uv[:5,1], speed[:5])

# root, ext = os.path.splitext(__file__)
mapfile = 'quiver'  + '.html'
# Create the map
mplleaflet.show(path=mapfile, crs=crs_out, tiles=('https://api.mapbox.com/styles/v1/jwasserman/cir51iqda0010bmnic1s5sb71/tiles/256/{z}/{x}/{y}?access_token=pk.eyJ1Ijoiandhc3Nlcm1hbiIsImEiOiJjaW9kNnRiaXUwNGh0dmFrajlqZ25wZnFsIn0.CU4YynqRJkmG1PwWDMBJSA', '<a href="https://mapbox.com/about/maps">© 2017 Mapbox</a> | <a href=https://www.openstreetmap.org/about">© OpenStreetMap</a>'))